In [1]:
import pandas as pd
import boto3
import numpy as np
from sklearn.datasets import dump_svmlight_file 
import sagemaker
import os

In [2]:
bucket = 'datascience-sagemaker-laurawindlin'


In [3]:
dados_brutos_arquivo = 'Case_cobranca.csv'

try:
    os.mkdir('dados')
except:
    pass

dados = pd.read_csv("s3://{}/{}".format(bucket, "Case_cobranca.csv"))
pd.set_option('display.max_rows', 20) 
dados

,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO
0,1,NOVO,CDC,M,26.0,0,227.0
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0
2,3,NOVO,PESSOAL,M,31.0,1,NaN
3,4,NOVO,PESSOAL,F,45.0,11,NaN
4,5,NOVO,CDC,F,25.0,0,8.0
...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0


In [4]:
dados['ALVO']   = dados['TEMP_RECUPERACAO'].apply(lambda x: 1 if x <= 90 else 0)dados

,COD,TIPO_CLIENTE,TIPO_EMPRESTIMO,CD_SEXO,IDADE,QTD_DIVIDAS,TEMP_RECUPERACAO,ALVO
0,1,NOVO,CDC,M,26.0,0,227.0,0
1,2,NOVO,RENEGOCIAÇÃO,M,39.0,0,37.0,1
2,3,NOVO,PESSOAL,M,31.0,1,NaN,0
3,4,NOVO,PESSOAL,F,45.0,11,NaN,0
4,5,NOVO,CDC,F,25.0,0,8.0,1
...,...,...,...,...,...,...,...,...
9812,9813,NOVO,CDC,F,38.0,1,23.0,1
9813,9814,NOVO,RENEGOCIAÇÃO,M,34.0,2,234.0,0
9814,9815,TOMADOR,CDC,NaN,30.0,4,13.0,1
9815,9816,INVESTIDOR,CDC,M,43.0,0,6.0,1


In [9]:
dados['CLIENTE_NOVO']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'NOVO' else 0)
dados['CLIENTE_INVESTIDOR']   = dados['TIPO_CLIENTE'].apply(lambda x: 1 if x == 'INVESTIDOR' else 0)             
dados['EMPRESTIMO_CDC']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'CDC' else 0)
dados['EMPRESTIMO_PESSOAL']   = dados['TIPO_EMPRESTIMO'].apply(lambda x: 1 if x == 'PESSOAL' else 0)
dados['SEXO_M']   = dados['CD_SEXO'].apply(lambda x: 1 if x == 'M' else 0)
dados['SEXO_M']   = dados['CD_SEXO'].apply(lambda x: 1 if x == 'M' else 0)
dados['QTD_DIVIDAS_NORM'] = dados['QTD_DIVIDAS'].apply(lambda x: 0. if np.isnan(x) else x/16) # normalização entre 0 e 1
dados['IDADE_NORM'] = dados['IDADE'].apply(lambda x: 18 if np.isnan(x) or x < 18 else x) # mínimo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: 76 if x > 76 else x) # máximo
dados['IDADE_NORM'] = dados['IDADE_NORM'].apply(lambda x: (x-18)/(76-18)) # normalização entre 0 e 1
dados = dados.drop(['COD', 'TIPO_CLIENTE', 'TIPO_EMPRESTIMO', 'CD_SEXO', 'IDADE', 'QTD_DIVIDAS', 'TEMP_RECUPERACAO'], axis=1)
dados

KeyError: 'TIPO_CLIENTE'

In [8]:
treinamento, validacao, teste = np.split(dados.sample(frac=1, random_state=88), [int(0.7 * len(dados)), int(0.9 * len(dados))]) 
len(dados),len(treinamento), len(validacao), len(teste)

(9817, 6871, 1964, 982)

In [10]:
treinamento.to_csv('dados/treinamento.csv', index=False, header=False)
validacao.to_csv('dados/validacao.csv', index=False, header=False)
teste.to_csv('dados/teste.csv', index=False)

In [12]:
boto3.Session().resource('s3').Bucket(bucket).Object('treinamento.csv').upload_file('dados/treinamento.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('validacao.csv').upload_file('dados/validacao.csv')
boto3.Session().resource('s3').Bucket(bucket).Object('teste.csv').upload_file('dados/teste.csv')